In [1]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import sobol_seq 
import random

In [2]:
template=pd.read_csv("Design Study Template_DTL3.csv", encoding="ISO-8859-1")
df=template.copy()
# print(df)
df.reset_index(inplace=True)  
if "index" in df.columns:
    df.drop(["index"], axis=1, inplace=True)
# print(df)
df.drop(df.index[1], inplace=True)
df.drop(df.columns[0:2], axis=1, inplace=True)

df.columns = df.iloc[0]
# print(df.columns)
df.drop(df.index[0], inplace=True)

#Define specific materials to insert into design templates
steel=df.at[2, " Material"]
aluminum=df.at[3, " Material"]
titanium=df.at[4, " Material"]

print(steel)
print(aluminum)
print(titanium)
df.drop(df.index, axis=0, inplace=True)
df.head()
# df.columns


 AISI 4130 Steel#^# annealed at 865C@SOLIDWORKS Materials$SOLIDWORKS Materials|AISI 4130 Steel#^# annealed at 865C|94$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|AISI 4130 Steel#^# annealed at 865C|94
 6061-T6 (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|6061-T6 (SS)|164$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|6061-T6 (SS)|164
 Ti-6Al-4VSolution treated and aged (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|Ti-6Al-4VSolution treated and aged (SS)|205$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|Ti-6Al-4VSolution treated and aged (SS)|205


,CS Length,BB Drop,Stack,SS E,ST Angle,BB OD,TT OD,HT OD,DT OD,CS OD,...,CSB Offset,SS Z,SS Thickness,CS Thickness,TT Thickness,BB Thickness,HT Thickness,ST Thickness,DT Thickness,DT Length


In [3]:
# def shufflethickness(df):

def geom_resample(minval, maxval, perc):
    maxval=np.full(np.shape(perc),maxval)
    return np.multiply(np.exp(np.multiply(np.log(np.divide(maxval, minval)),perc)), minval)

def shufflethickness(df):
    scalemin = .5
    scalemax = 10
    
    columns=["Wall thickness Bottom Bracket", "Wall thickness Top tube", "Wall thickness Head tube", "Wall thickness Down tube", "Wall thickness Chain stay", "Wall thickness Seat stay", "Wall thickness Seat tube"]
    indices=list(df.index)
#     for i in range(len(df.index)):
#         if i%10!=0:
#             indices.append(df.index[i])
    random.shuffle(indices)
    len_sobol= len(indices)
    sobolvec= sobol_seq.i4_sobol_generate(7, len_sobol)
    thicknesses=geom_resample(scalemin, scalemax, sobolvec)
    thicknesses=pd.DataFrame(thicknesses, columns=columns, index=indices)
    print(thicknesses)
    df.update(thicknesses)
    return df

In [5]:
biked=pd.read_csv("BIKED_reduced.csv", index_col=0)
biked_rethick=shufflethickness(biked)
# print(biked_rethick)

      Wall thickness Bottom Bracket  Wall thickness Top tube  \
3429                       2.236068                 2.236068   
493                        4.728708                 1.057371   
758                        1.057371                 4.728708   
1637                       1.537646                 1.537646   
4126                       6.876560                 6.876560   
...                             ...                      ...   
3142                       1.196924                 0.685534   
3457                       5.352806                 3.065803   
904                        2.531186                 1.449729   
3681                       0.565990                 6.483383   
3529                       0.593113                 0.654185   

      Wall thickness Head tube  Wall thickness Down tube  \
3429                  2.236068                  2.236068   
493                   4.728708                  1.057371   
758                   1.057371                  4.7

In [6]:
for col in df.columns:
    if col==" CS Length":
        df[col]=biked["CS textfield"]
    if col==" BB Drop":
        df[col]=biked["BB textfield"]
    if col==" Stack":
        df[col]=biked["Stack"]
    if col==" HT Angle":
        df[col]=biked["Head angle"]
    if col==" HT Length":
        df[col]=biked["Head tube length textfield"]
    if col==" SS E":
        df[col]=biked["Seat stay junction0"]
    if col==" ST Angle":
        df[col]=biked["Seat angle"]
    if col==" DT Length":
        df[col]=biked["DT Length"]
    if col==" ST Length":
        df[col]=biked["Seat tube length"]
    if col==" BB OD":
        df[col]=biked["BB diameter"]
    if col==" TT OD":
        df[col]=biked["ttd"]
    if col==" HT OD":
        df[col]=biked["Head tube diameter"]
    if col==" DT OD":
        df[col]=biked["dtd"]
    if col==" CS OD":
        df[col]=biked["csd"]
    if col==" SS OD":
        df[col]=biked["ssd"]
    if col==" ST OD":
        df[col]=biked["Seat tube diameter"]
        
    if col==" BB Thickness":
        df[col]=biked["Wall thickness Bottom Bracket"]
    if col==" TT Thickness":
        df[col]=biked["Wall thickness Top tube"]
    if col==" HT Thickness":
        df[col]=biked["Wall thickness Head tube"]
    if col==" DT Thickness":
        df[col]=biked["Wall thickness Down tube"]
    if col==" CS Thickness":
        df[col]=biked["Wall thickness Chain stay"]
    if col==" SS Thickness":
        df[col]=biked["Wall thickness Seat stay"]
    if col==" ST Thickness":
        df[col]=biked["Wall thickness Seat tube"]
        
    if col==" BB Length":
        df[col]=biked["BB length"]
    if col==" CS F":
        df[col]=biked["Chain stay position on BB"]
    if col==" SS Z":
        df[col]=biked["SSTopZOFFSET"]
    if col==" Material":
        df[col]=biked["MATERIAL"]
    if col==" HT UX":
        df[col]=biked["Head tube upper extension2"]
    if col==" ST UX":
        df[col]=biked["Seat tube extension2"]
    if col==" HT LX":
        df[col]=biked["Head tube lower extension2"]
    if col==" SSB Offset":
        df[col]=biked["SEATSTAYbrdgshift"]
    if col==" CSB Offset":
        df[col]=biked["CHAINSTAYbrdgshift"]
    if col==" Dropout Offset":
        df[col]=biked["Dropout spacing"]

invalid_bikes=[]
for idx in df.index:
    
  
    
    invalid=False
    if df.at[idx, " CS Length"]<=0:
        invalid=True
    if df.at[idx, " Stack"]<=0:
        invalid=True
    if df.at[idx, " ST Length"]<=0:
        invalid=True
    if df.at[idx, " HT Length"]<=0:
        invalid=True
    if df.at[idx, " DT Length"]<=0:
        invalid=True
    if df.at[idx, " BB Length"]<=0:
        invalid=True
    if df.at[idx, " HT Angle"]<=0:
        invalid=True
    if df.at[idx, " SS E"]<=0:
        invalid=True
    if df.at[idx, " SS Z"]<=0:
        invalid=True
    if df.at[idx, " SSB Offset"]<=0:
        invalid=True
    if df.at[idx, " CSB Offset"]<=0:
        invalid=True
    if df.at[idx, " ST Angle"]<=0:
        invalid=True
    if df.at[idx, " BB OD"]<=0:
        invalid=True
    if df.at[idx, " TT OD"]<=0:
        invalid=True
    if df.at[idx, " HT OD"]<=0:
        invalid=True
    if df.at[idx, " DT OD"]<=0:
        invalid=True
    if df.at[idx, " CS OD"]<=0:
        invalid=True
    if df.at[idx, " SS OD"]<=0:
        invalid=True
    if df.at[idx, " ST OD"]<=0:
        invalid=True
    if df.at[idx, " HT UX"]<=0:
        invalid=True
    if df.at[idx, " HT LX"]<=0:
        invalid=True
    if df.at[idx, " ST UX"]<=0:
        invalid=True
    if df.at[idx, " ST Angle"]<=0:
        invalid=True
    if df.at[idx, " ST Angle"]>=180:
        invalid=True
    if df.at[idx, " HT Angle"]>=180:
        invalid=True
    if df.at[idx, " HT Angle"]<=0:
        invalid=True
    if df.at[idx, " CS F"]<=0:
        invalid=True
    if df.at[idx, " Dropout Offset"]<=0:
        invalid=True
#         print(idx)
#         invalid=True
#     if (df.at[idx, " ST Length"]+df.at[idx, " ST UX"])*np.sin(df.at[idx, " ST Angle"]/180*np.pi)>=1700:
#         print(idx)
#         invalid=True
    if (df.at[idx, " CS F"]>df.at[idx, " BB Length"])/2:
        invalid=True
    if df.at[idx, " SS OD"] + df.at[idx, " ST OD"] < 2*df.at[idx, " SS Z"]:
        invalid=True
        
        
    #Set bridge diameter values based on whether bridges are on. If bridges are off, set diameter to 0.
    if biked.at[idx, "SEATSTAYbrdgCheck"]==1:
        df.at[idx, " SSB OD"]=biked.at[idx, "SEATSTAYbrdgdia1"]
        if biked.at[idx, "SEATSTAYbrdgdia1"]<0:
            invalid=True
    else:
        df.at[idx, " SSB OD"]=0
    if biked.at[idx, "CHAINSTAYbrdgCheck"]==1:
        df.at[idx, " CSB OD"]=biked.at[idx, "CHAINSTAYbrdgdia1"]
        if biked.at[idx, "CHAINSTAYbrdgdia1"]<0:
            invalid=True
    else:
        df.at[idx, " CSB OD"]=0
    
    #Set Material names
    if df.at[idx, " Material"]=="STEEL":
        df.at[idx, " Material"]=steel
    elif df.at[idx, " Material"]=="TITANIUM":
        df.at[idx, " Material"]=titanium
#     if df.at[idx, " Material"]=="ALUMINIUM":
    else:
        df.at[idx, " Material"]=aluminum

        
    if invalid==True:
        invalid_bikes.append(idx)
        


#Scale values from mm to meters, excluding any angles and material parameter        
columns=list(df.columns)
columns.remove(" Material")
columns.remove(" HT Angle")
columns.remove(" ST Angle")
scaleddf=df[columns].divide(1000)
scaleddf[" Material"]=df[" Material"]
scaleddf[" HT Angle"]=df[" HT Angle"]
scaleddf[" ST Angle"]=df[" ST Angle"]
scaleddf=scaleddf[df.columns]

#Save the dataframe before dropping invalid bikes for future use in validity classification
scaleddf.to_csv("biked_predrop.csv")
print(invalid_bikes)
scaleddf.drop(invalid_bikes, inplace=True)



[3, 6, 11, 17, 20, 23, 30, 31, 57, 68, 109, 110, 122, 158, 177, 188, 189, 195, 199, 234, 240, 248, 253, 289, 290, 325, 329, 332, 336, 345, 353, 359, 372, 390, 391, 398, 403, 410, 429, 432, 442, 453, 455, 503, 505, 516, 525, 528, 542, 553, 579, 667, 671, 713, 749, 771, 844, 858, 918, 921, 947, 987, 990, 1013, 1040, 1042, 1048, 1062, 1065, 1110, 1151, 1154, 1169, 1183, 1196, 1218, 1232, 1233, 1253, 1284, 1299, 1333, 1336, 1344, 1345, 1346, 1347, 1355, 1356, 1366, 1368, 1369, 1377, 1382, 1387, 1407, 1413, 1416, 1434, 1435, 1453, 1464, 1472, 1490, 1496, 1555, 1556, 1669, 1693, 1700, 1736, 1761, 1765, 1771, 1783, 1787, 1789, 1790, 1791, 1793, 1801, 1805, 1810, 1823, 1829, 1863, 1873, 1891, 1908, 1926, 1956, 1957, 1962, 1988, 2016, 2036, 2075, 2110, 2148, 2162, 2192, 2259, 2260, 2268, 2313, 2326, 2405, 2422, 2428, 2435, 2472, 2482, 2518, 2596, 2606, 2641, 2643, 2674, 2675, 2676, 2691, 2756, 2808, 2838, 2839, 2866, 2881, 2883, 2930, 2931, 2939, 2968, 2995, 3018, 3032, 3095, 3102, 3117, 3119, 

In [7]:
scaleddf.to_csv("Processed_structural_data.csv")

In [8]:
augdf=scaleddf
augdf=augdf[3400:3600]

In [10]:
for idx in augdf.index:
    if augdf.at[idx, " CS F"]==augdf.at[idx, " BB Length"]/2:
        augdf.at[idx, " CS F"]=augdf.at[idx, " CS F"]-.0001
        print("Adjusting CS F for model " + str(idx) + " by 0.0001 since it is half of BB length and would cause model to lock!")


template=pd.read_csv("Design Study Template_DTL3.csv", encoding="ISO-8859-1")
# if "index" in template.columns:
#     template.drop(["index"], axis=1, inplace=True)
newindex=augdf.columns.insert(0,"Status")
augdf["Status"]=np.full(len(augdf.index), 1)
augdf=augdf[newindex]
augdf.index=["Bike" + str(i) for i in augdf.index]
# print(template)
template.reset_index(inplace=True)  
units=template.iloc[1]

#Old code to drop "Level_0", replace this by dropping entries from top of units until reaching correct length
#     units.drop(["level_0"], axis=0, inplace=True)

while len(units.index)>len(augdf.columns):
    units.drop([units.index[0]], axis=0, inplace=True)
units.index=augdf.columns
units.rename("Units", inplace=True)
augdf=augdf.append(units)
newindices= augdf.index[:-1].insert(0, augdf.index[-1])
augdf=augdf.reindex(newindices)
augdf.columns.rename("Parameters", inplace=True)

augdf=augdf.T.reset_index().T
cols=[len(augdf.index)-2, len(augdf.columns)-1] + [""]*(len(augdf.columns)-2)
# print(cols)
augdf.columns=cols
augdf.columns.rename("Design Set", inplace=True)
# print(units)
# print(augdf)

<ipython-input-10-85bf77ba1d72>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augdf["Status"]=np.full(len(augdf.index), 1)
C:\Users\Lyle\anaconda3\envs\tf\lib\site-packages\pandas\core\series.py:4443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
augdf.to_csv("generated_param_study_template.csv", index_label="Design Set")